In [2]:
%load_ext autoreload
%autoreload 2
from config.datasets import dataset_configs
from data.datasets import StormDataset, ExperimentalDataSet

dataset ='20220729_nup'

cfg = dataset_configs[dataset]['fov3_0']
ds = StormDataset(cfg, normalize_psf=True, lazy=True)
ds.csv_data = ds.csv_data[ds.csv_data['sigma [nm]'] > 20]
ds.prepare_data()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Reading img...
Loading /home/miguel/Projects/uni/data/smlm_3d/20220729_nup/28_07_2022_sample/prime95b/A2/FOV3/storm_1/storm_1_MMStack_Default_1_cropped.ome.tif
Loading /home/miguel/Projects/uni/data/smlm_3d/20220729_nup/28_07_2022_sample/prime95b/A2/FOV3/storm_1/storm_1_MMStack_Default_1_cropped.csv
425133 emitters before filtering


  0%|          | 126/365224 [00:00<04:50, 1258.15it/s]

365224 emitters after borders


100%|██████████| 365224/365224 [03:38<00:00, 1671.22it/s]


Clustering emitters...
Initial: N images 365224 - DF: (365224, 10)
Final: images 1729 - DF: (1729, 12)


In [ ]:
%load_ext autoreload
%autoreload 2
from data.visualise import scatter_3d, scatter_yz, show_psf_axial, plot_with_sphere, grid_psfs
from data.datasets import ExperimentalDataSet, TrainingDataSet

from config.datasets import dataset_configs
from data.estimate_offset import remove_bg
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import ipywidgets as widgets
from IPython.display import display, clear_output
from functools import partial
from PIL import Image

cfgs = dataset_configs['20220425_Miguel']

psfs = []
for cfg in cfgs:
    if cfg != 'training_20nm':
        continue
    cfg = cfgs['training_20nm']
    ds = TrainingDataSet(cfg, fit_plane_z=False, transform_data=False, z_range=10000, lazy=True, split_data=False, add_noise=False, filter_emitters_proximity=True)
    ds.prepare_debug()
    for i in range(ds.csv_data.shape[0]):
        psfs.append(ds.debug_emitter(i, 10000)[0])
        print(len(psfs))
    del ds

psfs = np.array(psfs)
print(psfs.shape)

In [ ]:
print(psfs.shape)

In [ ]:
from pyotf.otf import HanserPSF, apply_aberration, apply_named_aberration

kwargs = dict(
    wl=647,
    na=1.3,
    ni=1.51,
    res=106,
    zres=10,
    size=32,
    zsize=200,
    vec_corr="none",
    condition="none",
)
psf = HanserPSF(**kwargs)
psf = apply_aberration(psf, np.array([0, 0, 0, 0, 0]), np.array([0, 0, 0, 0, 1]))

blank_psf = psf.PSFi

print(np.argmax(blank_psf.max(axis=(1,2))))



In [ ]:
import numpy as np
from experiments.noise.noise_psf import generate_noisy_psf
fake_psfs = []

blank_psf = blank_psf / blank_psf.max()

snrs = []
n_psfs = 100
for _ in range(n_psfs):
    nsr = np.random.uniform(0, 0.25)
    fake_psf = generate_noisy_psf(blank_psf, nsr)
    fake_psfs.append(fake_psf)
    snr =  fake_psf.max() / np.median(fake_psf)
    snrs.append(snr)
    

psfs = np.array(fake_psfs)
    

In [ ]:
import numpy as np
from data.estimate_offset import norm_zero_one

def remove_bg(img):
    img = norm_zero_one(img)
    if img.ndim == 3:
        bg_level = img.max(axis=(1,2)).min()
    else:
        bg_level = img.min()
    mult = 1.2
    img = img - (bg_level * mult)
    img[img<0] = 0
    img = norm_zero_one(img)
    return img
psfs = np.array([remove_bg(psf) for psf in psfs])

In [ ]:
import pandas as pd

from data.estimate_offset import get_sharpness
from scipy.interpolate import UnivariateSpline
plt.rcParams['figure.figsize'] = [10, 5]

sharpness = np.array([get_sharpness(psf) for psf in psfs])
n_psfs = psfs.shape[0]
res = []

sub_x = np.linspace(0, psfs.shape[1], 1000, endpoint=True)

    
def fit_cubic_spline(y, x, sub_x, s=0.05):
    cs = UnivariateSpline(x, list(y), k=3, s=s)
    print(cs(x))
    # TODO potentially use this to exclude beads?
    # evaluate sum of squared second derivative as a measurement of curve smoothness
    smoothness = sum(np.power(cs(sub_x, 2), 2))

    if smoothness > 5:
        plt.plot(x, norm_zero_one(cs(x)))
        raise EnvironmentError('Failed to fit bead accurately.')

    return cs(sub_x)


for std in [5, 10, 15, 20, 25, 30, 35, 40]:
    
    for i in range(n_psfs):
        for window in range(5, sharpness.shape[1]//2, 5):
            smooth_data = pd.Series(sharpness[i]).rolling(window=window, win_type='gaussian', center=True).mean(std=std)
            hr_smooth_data = fit_cubic_spline(smooth_data, smooth_data.index, sub_x)
            
            plt.plot(smooth_data, label=str(window))
#             plt.plot(sharpness[i], label='raw')
            plt.plot(hr_smooth_data+1, label='high_res_smooth')
            print(hr_smooth_data)
            plt.legend()
            plt.show()
            res.append((window, np.argmax(smooth_data), snrs[i], std))
            break
        break
    break
    #     plt.legend()
    #     plt.show()
#     if i2 == 95 and std > 5:
#         show_psf_axial(psfs[i])
    #     break
    #     snr = fake_psfs[i].max() / np.median(fake_psfs[i])
    #     plt.title(f'{snr} - {1/nsrs[i]}')
    #     show_psf_axial(fake_psfs[i])


In [ ]:
x, y, snr, std = zip(*res)
import pandas as pd
import seaborn as sns

df = pd.DataFrame.from_dict({'smooth': x, 'pred': y, 'snr': snr, 'std': std})


print('a', df)
# df = df[df['snr']> 10] 
print('b', df.shape)

sns.scatterplot(data=df, x='smooth', y='pred', hue='snr')
plt.show()

means = df.groupby(['smooth', 'std']).std()
print(means['pred'].idxmin())
print(means)